In [5]:
mutable struct Monkey
    items
    operation
    test
    iftrue
    iffalse
    n_inspections
end

function parse_operation(parts)
    op = parts[2]
    val = parts[3]
    if val == "old"
        if op == "*"
            return x -> x * x
        elseif op == "+"
            return x -> x + x
        end
    else
        if op == "*"
            return x -> x * parse(Int, val)
        elseif op == "+"
            return x -> x + parse(Int, val)
        end
    end
end

function read_input(filename)
    lines = open(filename, "r") do f
        [strip(line) for line in readlines(f)]
    end
    monkeys = []
    for i = 1:7:length(lines)
        items_raw = split(lines[i + 1], " ")[3:end]
        items = parse.(Int, [s[1:end - 1] for s in items_raw[1:end - 1]])
        push!(items, parse(Int, items_raw[end]))
        operation = parse_operation(split(lines[i + 2], " ")[4:end])
        test = parse(Int, split(lines[i + 3], " ")[end])
        iftrue = parse(Int, split(lines[i + 4], " ")[end]) + 1
        iffalse = parse(Int, split(lines[i + 5], " ")[end]) + 1
        push!(monkeys, Monkey(items, operation, test, iftrue, iffalse, 0))
    end
    monkeys
end

function throw(monkeys, n_rounds)
    for i = 1:n_rounds
        for monkey in monkeys
            while length(monkey.items) > 0
                monkey.n_inspections += 1
                item = popfirst!(monkey.items)
                worry = monkey.operation(item)
                worry = worry ÷ 3
                to_monkey = worry % monkey.test == 0 ? monkey.iftrue : monkey.iffalse
                push!(monkeys[to_monkey].items, worry)
            end
        end
    end
end

function throw2(monkeys, n_rounds)
    tests = [monkey.test for monkey in monkeys]
    for monkey in monkeys
        monkey.items = [[item % test for test in tests] for item in monkey.items]
    end
    for i = 1:n_rounds
        for (i, monkey) in enumerate(monkeys)
            while length(monkey.items) > 0
                monkey.n_inspections += 1
                item = popfirst!(monkey.items)
                worry = [monkey.operation(item_rem) % test for (item_rem, test) in zip(item, tests)]
                to_monkey = worry[i] == 0 ? monkey.iftrue : monkey.iffalse
                push!(monkeys[to_monkey].items, worry)
            end
        end
    end
end

function throw3(monkeys, n_rounds)
    m = prod(monkey.test for monkey in monkeys)
    for i = 1:n_rounds
        for monkey in monkeys
            while length(monkey.items) > 0
                monkey.n_inspections += 1
                item = popfirst!(monkey.items)
                worry = monkey.operation(item)
                worry = worry % m
                to_monkey = worry % monkey.test == 0 ? monkey.iftrue : monkey.iffalse
                push!(monkeys[to_monkey].items, worry)
            end
        end
    end
end

function runit(filename; part2=false)
    monkeys = read_input(filename)
    if !part2
        throw(monkeys, 20)
    else
        throw2(monkeys, 10000)
    end
    max_inspections = sort([monkey.n_inspections for monkey in monkeys], rev=true)[1:2]
    prod(max_inspections)
end

runit (generic function with 1 method)

In [6]:
runit("11_input.txt")

98280

In [8]:
runit("11_input.txt", part2=true)

17673687232